In [ ]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# DEPENDENCIES
!git clone https://github.com/PKU-YuanGroup/LanguageBind

Cloning into 'LanguageBind'...
remote: Enumerating objects: 1356, done.
remote: Counting objects: 100% (380/380), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 1356 (delta 274), reused 284 (delta 197), pack-reused 976
Receiving objects: 100% (1356/1356), 18.98 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (730/730), done.


In [ ]:
%cd LanguageBind

/content/LanguageBind


In [ ]:
!pip install -q torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!pip install -q einops
!pip install -q peft
!pip install -q transformers==4.31.0
!pip install -q decord
!pip install -q pytorchvideo
!pip install -q urlextract
!pip install -q moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 905.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 94.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1+cu116 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1+cu116 which is incompatible.


In [ ]:
import urllib
import pandas as pd
import moviepy
from moviepy import editor

import torch
from languagebind import LanguageBindAudio, LanguageBindAudioTokenizer, LanguageBindAudioProcessor

In [ ]:
pretrained_ckpt = 'LanguageBind/LanguageBind_Audio'  # also 'LanguageBind/LanguageBind_Audio'
model_a = LanguageBindAudio.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir').to('cuda')
tokenizer_a = LanguageBindAudioTokenizer.from_pretrained(pretrained_ckpt, cache_dir='./cache_dir',device='cuda')
audio_process = LanguageBindAudioProcessor(model_a.config, tokenizer_a)

In [ ]:
def encode_audio(url):
  if('/vid/' not in url):
    print("not a video")
    return
  urllib.request.urlretrieve(url,"vid.mp4")
  print("Got video")

  video = moviepy.editor.VideoFileClip("vid.mp4")
  audio = video.audio
  if audio:
    audio.write_audiofile("audio.wav")
    print("Audio extracted successfully")
  else:
    print("No audio found in the video")
    return

  data = audio_process(["/content/LanguageBind/audio.wav"], ["text"], return_tensors='pt').to('cuda')
  print("Processed audio")
  #data = {key: val.to(device) for key, val in data.items()}  # Move data to GPU
  model_a.eval()

  with torch.no_grad():
      out = model_a(**data)
      return out.image_embeds  # Return image embeddings
  print("Got embeddings")

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/hellod audio.csv")

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50001 entries, 0 to 50000
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0.14     50001 non-null  int64 
 1   Unnamed: 0.13     50001 non-null  int64 
 2   Unnamed: 0.12     50001 non-null  int64 
 3   Unnamed: 0.11     50001 non-null  int64 
 4   Unnamed: 0.10     50001 non-null  int64 
 5   Unnamed: 0.9      50001 non-null  int64 
 6   Unnamed: 0.8      50001 non-null  int64 
 7   Unnamed: 0.7      50001 non-null  int64 
 8   Unnamed: 0.6      50001 non-null  int64 
 9   Unnamed: 0.5      50001 non-null  int64 
 10  Unnamed: 0.4      50001 non-null  int64 
 11  Unnamed: 0.3      50001 non-null  int64 
 12  Unnamed: 0.2      50001 non-null  int64 
 13  Unnamed: 0.1      50001 non-null  int64 
 14  Unnamed: 0        50001 non-null  int64 
 15  id                50001 non-null  int64 
 16  date              50001 non-null  object
 17  likes       

In [ ]:
train_df['audio'][33431]

In [ ]:
count_check = 0
for i in range(15000, 20000):
  print("*************************", i, "*****************************")
  url = train_df["single_media"][i]
  op = encode_audio(url)
  train_df["audio"][i] = op
  count_check+=1
  if count_check % 50 == 0: # Periodically save
    train_df.to_csv("/content/drive/MyDrive/hellod audio.csv") 

In [ ]:
train_df.to_csv("/content/drive/MyDrive/hellod audio.csv")

In [ ]:
d_df=pd.read_csv('/content/drive/MyDrive/hellod audio.csv')

In [ ]:
d_df['audio'][34184]

"tensor([[ 0.0385, -0.0512,  0.0173, -0.0307, -0.0731,  0.0161,  0.0181,  0.0201,\n         -0.0953,  0.0260, -0.0435,  0.0383,  0.0138, -0.0149,  0.0108,  0.0483,\n          0.0459, -0.0354,  0.0141,  0.0197,  0.0163, -0.0126,  0.0277, -0.0554,\n          0.0354, -0.0459, -0.0420,  0.0107,  0.0823,  0.0569, -0.0092, -0.0258,\n          0.0039,  0.0004, -0.0252, -0.0313,  0.0049, -0.0840, -0.0521,  0.0810,\n          0.0401,  0.0241,  0.0220, -0.0068, -0.0115,  0.0799,  0.0037, -0.0140,\n         -0.0306,  0.0116,  0.0058, -0.0372, -0.0275, -0.0057,  0.0962, -0.0281,\n         -0.0060, -0.0313,  0.0164,  0.0019, -0.0346,  0.0093,  0.0243,  0.0140,\n         -0.0125, -0.0343, -0.0131,  0.0197,  0.0032,  0.0341,  0.0070, -0.0507,\n         -0.0647,  0.0048, -0.0245, -0.0268,  0.0219,  0.0111, -0.0536,  0.0932,\n          0.0091, -0.0244, -0.0488, -0.0376,  0.0016, -0.0516, -0.0068,  0.0506,\n          0.0639, -0.0720, -0.0184,  0.0180,  0.0322, -0.0142,  0.0251, -0.0199,\n         -0.043